# Script for creating database and DW, its tables and pushing data to these tables

### 1. Running sub-scripts

In [3]:
%run Weather.ipynb
%run Crime_final.ipynb

### Configurations

In [4]:
import pyodbc
import pandas as pd
from warnings import simplefilter

WEATHER_FILE = 'Weather_final.csv'
CRIME_FILE = 'CRIME_FILE.csv'

SERVER = 'localhost'
DATABASE = 'Miniprojekti2'
DATAWAREHOUSE = 'Miniprojekti2DW'
USER = 'sa'
PWD = 'Password1'

simplefilter(action='ignore', category=UserWarning)

### 2. Create DB if not exist

In [5]:
try:
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';UID='+USER+';PWD='+ PWD, autocommit=True)
    cursor = cnxn.cursor()
    cursor.execute("CREATE DATABASE Miniprojekti2")
    cnxn.commit()
    cursor.close()
    cnxn.close()
except:
    print('Database already exists!')

### 3. Create Weather table if not exist

In [6]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()
try:
    cursor.execute("""
    CREATE TABLE Weather(
        DateKey int Primary Key,
        Temperature float,
        Humidity float,
        Rainfall float,
        Clouds float,
        Wind float,
        Weather_time datetime)
    """)

except:
    print('Table already exists!')
    cursor.execute("TRUNCATE TABLE Weather")

cnxn.commit()
cursor.close()
cnxn.close()

### 4. Load data into Weather table

In [7]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()
df_weather = pd.read_csv(WEATHER_FILE)
for index, row in df_weather.iterrows():
    cursor.execute("INSERT INTO dbo.Weather values(?,?,?,?,?,?,?)",
                   row['DateKey'],
                   row['Temperature'],
                   row['Humidity'],
                   row['Rainfall'],
                   row['Clouds'],
                   row['Wind'],
                   row['Weather_time']
                   )
    cnxn.commit()
cursor.close()
cnxn.close()

### 5. Create Crime table if not exist

In [8]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()
try:
    cursor.execute("""
    CREATE TABLE Crimes(
        CrimeID int Primary Key,
        IncidentCategory varchar(250),
        Neighborhood varchar(250),
        Datetime datetime,
        Hour int,
        Weather int)
    """)

except:
    print('Table already exists!')
    cursor.execute("TRUNCATE TABLE Crimes")

cnxn.commit()
cursor.close()
cnxn.close()

### 6. Load data into Crime table

In [9]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()
df = pd.read_csv(CRIME_FILE, sep=';')
for index, row in df.iterrows():
    cursor.execute('INSERT INTO dbo.Crimes values(?,?,?,?,?,?)',
                   index,
                   row['incidentCategory'],
                   row['neighborhood'],
                   row['datetime'],
                   row['hour'],
                   row['weather']
                   )
    cnxn.commit()
cursor.close()
cnxn.close()

### 7. Create DW if not exist

In [10]:
try:
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';UID='+USER+';PWD='+ PWD, autocommit=True)
    cursor = cnxn.cursor()
    cursor.execute("CREATE DATABASE Miniprojekti2DW")
    cnxn.commit()
    cursor.close()
    cnxn.close()
except:
    print('Database already exists!')

### 8. Create FactCrimes table if not exist

In [11]:
cnxn_dest = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATAWAREHOUSE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn_dest.cursor()
try:
    cursor.execute("""CREATE TABLE FactCrimes(
                        CrimeID int Primary Key,
                        IncidentCategory varchar(250),
                        Neighborhood varchar(250),
                        DateKey int,
                        Temperature float,
                        Humidity float,
                        Rainfall float,
                        Clouds float,
                        Wind float)
    """)
except:
    print('Table already exists!')
    cursor.execute("TRUNCATE TABLE FactCrimes")
    
cnxn_dest.commit()
cursor.close()
cnxn_dest.close()

### 9. Transform data for DW

In [12]:
cnxn_src = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)

df_dw = pd.read_sql("""
SELECT * 
FROM Crimes AS C
FULL JOIN Weather AS W ON C.Weather = W.DateKey
WHERE Weather BETWEEN 2022092501 AND 2023030712
""", con=cnxn_src)

# Set CrimeID as index
df_dw.set_index(['CrimeID'], inplace=True)
df_dw.drop(columns=['Weather','Weather_time'], inplace=True)
df_dw.to_csv('FactCrimes.csv')
df_dw.head()

,IncidentCategory,Neighborhood,Datetime,Hour,DateKey,Temperature,Humidity,Rainfall,Clouds,Wind
CrimeID,,,,,,,,,,
16843,Larceny Theft,Mission,2022-09-25 01:00:00,1,2022092501,15.5,97.0,0.0,0.0,2.2
16844,Motor Vehicle Theft,Hayes Valley,2022-09-25 01:00:00,1,2022092501,15.5,97.0,0.0,0.0,2.2
16845,Fraud,Mission,2022-09-25 01:00:00,1,2022092501,15.5,97.0,0.0,0.0,2.2
16846,Larceny Theft,Chinatown,2022-09-25 01:00:00,1,2022092501,15.5,97.0,0.0,0.0,2.2
16847,Malicious Mischief,Financial District/South Beach,2022-09-25 01:19:00,1,2022092501,15.5,97.0,0.0,0.0,2.2


### 10. Load FactCrimes

In [13]:
cnxn_dest = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATAWAREHOUSE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn_dest.cursor()

for index,row in df_dw.iterrows():
    cursor.execute('INSERT INTO dbo.FactCrimes values (?,?,?,?,?,?,?,?,?)',
                   index,
                   row['IncidentCategory'],
                   row['Neighborhood'],
                   row['DateKey'],
                   row['Temperature'],
                   row['Humidity'],
                   row['Rainfall'],
                   row['Clouds'],
                   row['Wind'])
    cnxn_dest.commit()
cursor.close()
cnxn_dest.close()

### 11. Create DimDate table if not exist

In [14]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATAWAREHOUSE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()
try:
    cursor.execute("""
    CREATE TABLE DimDate(
        DateKey int Primary Key,
        Datetime datetime,
        Year int,
        Month int,
        Day int,
        Hour int)
    """)

except:
    print('Table already exists!')
    cursor.execute("TRUNCATE TABLE DimDate")
    
cnxn.commit()
cursor.close()
cnxn.close()

### 12. Generate DimDate

In [15]:
%run DimDate.ipynb

### 13. Load DimDate into DW

In [16]:
df_dd = pd.read_csv('DimDate.csv')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATAWAREHOUSE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()

for index, row in df_dd.iterrows():
    cursor.execute("INSERT INTO DimDate values(?,?,?,?,?,?)", 
                   int(row['DateKey']), 
                   row['Datetime'], 
                   int(row['Year']), 
                   int(row['Month']), 
                   int(row['Day']), 
                   int(row['Hour']))
    cnxn.commit()
    
cursor.close()
cnxn.close()